In [1]:
import pandas as pd

In [59]:
data = pd.read_csv('D:\google_earth_engine\google_erath_engine_era5\Bio\data.csv')

In [60]:
def _rename_columns(data):
    # rename columns
    data.rename(columns={'temperature_2m_MEAN': 'Temp',
                                'temperature_2m_max_MEAN': 'Tmax',
                                'temperature_2m_min_MEAN': 'Tmin',
                                'total_precipitation_sum_MEAN': 'Prec',
                                'STATE':'ID'}, inplace=True)

def _convert_schema(data):
    # conver Date to datetime type
    data['Date'] = pd.to_datetime(data['Date'])

    data['Month'] = data['Date'].dt.month

    data['Year'] = data['Date'].dt.year

In [61]:
_rename_columns(data)
_convert_schema(data)

In [80]:
# Group by year and month to calculate the average temperature and total precipitation per month
data.sort_values(by=['ID', 'Date'], inplace=True)

data['Temp'] = data['Temp'] - 273.15
data['Tmax'] = data['Tmax'] - 273.15
data['Tmin'] = data['Tmin'] - 273.15

monthly_data = data.groupby(['ID', 'Year', 'Month']).agg({
    'Temp': 'mean',
    'Tmin': 'min',
    'Tmax': 'max',
    'Prec': 'sum'
}).reset_index()

# bio01: Average annual temperature
monthly_data['Bio01'] = monthly_data['Temp']

# bio02: Annual temperature range (maximum temperature of the year minus minimum temperature of the year)
monthly_data['Bio02'] = monthly_data['Tmax'] - monthly_data['Tmin']

# bio04: Isothermality index (annual variation of temperature, standard deviation of the highest and lowest temperatures of the year)
yearly_temperatures = data.groupby(['ID', 'Year'])['Temp'].agg('std').reset_index()
monthly_data = pd.merge(monthly_data, yearly_temperatures, on=['ID', 'Year'], suffixes=('', '_std'))
monthly_data['Bio04'] = monthly_data['Temp_std']

# bio05: Highest temperature of the warmest month
max_temp_warmest_month = monthly_data.groupby(['ID', 'Year'])['Tmax'].max().reset_index()
monthly_data = pd.merge(monthly_data, max_temp_warmest_month, on=['ID', 'Year'], suffixes=('', '_max_warm'))
monthly_data['Bio05'] = monthly_data['Tmax_max_warm']

# bio06: Lowest temperature of the coldest month
max_temp_warmest_month = monthly_data.groupby(['ID', 'Year'])['Tmin'].min().reset_index()
monthly_data = pd.merge(monthly_data, max_temp_warmest_month, on=['ID', 'Year'], suffixes=('', '_min_warm'))
monthly_data['Bio06'] = monthly_data['Tmin_min_warm']


# bio07: Annual temperature difference
monthly_data['Bio07'] = monthly_data['Bio05'] - monthly_data['Bio06']


# bio03: Isothermality (constant temperature measure)
monthly_data['Bio03'] = (monthly_data['Bio02'] / monthly_data['Bio07']) * 100


# bio08: Average temperature for the month with the highest cumulative precipitation over 3 consecutive months
# First calculate the cumulative precipitation for each month

rolling_data = data
rolling_data['cumulative_precipitation'] = rolling_data.groupby('ID')['Prec'].cumsum()

rolling_data['max_precip_3months'] = rolling_data.groupby('ID')['cumulative_precipitation'].transform(lambda x: x.rolling(window=3, min_periods=1).max())
final_data = rolling_data.drop_duplicates(subset=['ID', 'Year', 'Month'], keep='last')[['ID', 'Year', 'Month', 'max_precip_3months']]

# Find the month with the highest cumulative precipitation over 3 consecutive months
# max_rain_month = monthly_data.groupby(['ID', 'Year'])['cumulative_precipitation'].rolling(window=3).max().reset_index()


monthly_data = monthly_data.merge(final_data, on=['ID', 'Year', 'Month'], how='left')
monthly_data.rename(columns={'max_precip_3months': 'Bio08'}, inplace=True)


# bio09: Average temperature of the driest quarter
rolling_data = data
rolling_data['cumulative_precipitation'] = rolling_data.groupby('ID')['Prec'].cumsum()

rolling_data['min_precip_3months'] = rolling_data.groupby('ID')['cumulative_precipitation'].transform(lambda x: x.rolling(window=3, min_periods=1).min())
final_data = rolling_data.drop_duplicates(subset=['ID', 'Year', 'Month'], keep='last')[['ID', 'Year', 'Month', 'min_precip_3months']]

monthly_data = monthly_data.merge(final_data, on=['ID', 'Year', 'Month'], how='left')
monthly_data.rename(columns={'min_precip_3months': 'Bio09'}, inplace=True)

# bio10: Average temperature of the warmest quarter
# Find the month with the highest average temperature over 3 consecutive months
rolling_data = data
rolling_data['cumulative_temp'] = rolling_data.groupby('ID')['Temp'].cumsum()

rolling_data['max_temp_3months'] = rolling_data.groupby('ID')['cumulative_temp'].transform(lambda x: x.rolling(window=3, min_periods=1).max())
final_data = rolling_data.drop_duplicates(subset=['ID', 'Year', 'Month'], keep='last')[['ID', 'Year', 'Month', 'max_temp_3months']]


monthly_data = monthly_data.merge(final_data, on=['ID', 'Year', 'Month'], how='left')
monthly_data.rename(columns={'max_temp_3months': 'Bio10'}, inplace=True)

# bio11: Average temperature of the coldest quarter
# Find the month with the lowest average temperature over 3 consecutive months
rolling_data = data
rolling_data['cumulative_temp'] = rolling_data.groupby('ID')['Temp'].cumsum()

rolling_data['min_temp_3months'] = rolling_data.groupby('ID')['cumulative_temp'].transform(lambda x: x.rolling(window=3, min_periods=1).min())
final_data = rolling_data.drop_duplicates(subset=['ID', 'Year', 'Month'], keep='last')[['ID', 'Year', 'Month', 'min_temp_3months']]

monthly_data = monthly_data.merge(final_data, on=['ID', 'Year', 'Month'], how='left')
monthly_data.rename(columns={'min_temp_3months': 'Bio11'}, inplace=True)

# bio12: Annual precipitation
annual_precipitation = data.groupby(['ID', 'Year'])['Prec'].sum().reset_index()
monthly_data = pd.merge(monthly_data, annual_precipitation, on=['ID', 'Year'], suffixes=('', '_annual_precipitation'))
monthly_data['Bio12'] = monthly_data['Prec_annual_precipitation']

# bio13: Precipitation of the wettest month
max_precipitation_wettest_month = monthly_data.groupby(['ID', 'Year'])['Prec'].max().reset_index()
monthly_data = pd.merge(monthly_data, max_precipitation_wettest_month, on=['ID', 'Year'], suffixes=('', '_max_precipitation_wettest_month'))
monthly_data['Bio13'] = monthly_data['Prec_max_precipitation_wettest_month']

# bio14: Precipitation in driest month
min_precipitation_driest_month = monthly_data.groupby(['ID', 'Year'])['Prec'].min().reset_index()
monthly_data = pd.merge(monthly_data, min_precipitation_driest_month, on=['ID', 'Year'], suffixes=('', '_min_precipitation_driest_month'))
monthly_data['Bio14'] = monthly_data['Prec_min_precipitation_driest_month']

# bio15: Precipitation Seasonality
precipitation_seasonality = data.groupby(['ID', 'Year'])['Prec'].std() / data.groupby(['ID', 'Year'])['Prec'].mean() * 100
precipitation_seasonality = precipitation_seasonality.reset_index(name='Bio15')
monthly_data = pd.merge(monthly_data, precipitation_seasonality, on=['ID', 'Year'])

# bio16: Precipitation during the wettest quarter
max_precipitation_wettest_quarter = monthly_data.groupby(['ID', 'Year'])['Prec'].rolling(window=3).sum().groupby(['ID', 'Year']).max().reset_index()
monthly_data = pd.merge(monthly_data, max_precipitation_wettest_quarter, on=['ID', 'Year'], suffixes=('', '_max_precipitation_wettest_quarter'))
monthly_data['Bio16'] = monthly_data['Prec_max_precipitation_wettest_quarter']

# bio17: Precipitation during the driest quarter
min_precipitation_driest_quarter = monthly_data.groupby(['ID', 'Year'])['Prec'].rolling(window=3).sum().groupby(['ID', 'Year']).min().reset_index()
monthly_data = pd.merge(monthly_data, min_precipitation_driest_quarter, on=['ID', 'Year'], suffixes=('', '_min_precipitation_driest_quarter'))
monthly_data['Bio17'] = monthly_data['Prec_min_precipitation_driest_quarter']

# bio18: Precipitation during the warmest quarter
max_precipitation_warmest_quarter = monthly_data.groupby(['ID', 'Year'])['Prec'].rolling(window=3).sum().groupby(['ID', 'Year']).max().reset_index()
monthly_data = pd.merge(monthly_data, max_precipitation_warmest_quarter, on=['ID', 'Year'], suffixes=('', '_max_precipitation_warmest_quarter'))
monthly_data['Bio18'] = monthly_data['Prec_max_precipitation_warmest_quarter']

# bio19: Precipitation during the coldest quarter
min_precipitation_coldest_quarter = monthly_data.groupby(['ID', 'Year'])['Prec'].rolling(window=3).sum().groupby(['ID', 'Year']).min().reset_index()
monthly_data = pd.merge(monthly_data, min_precipitation_coldest_quarter, on=['ID', 'Year'], suffixes=('', '_min_precipitation_coldest_quarter'))
monthly_data['Bio19'] = monthly_data['Prec_min_precipitation_coldest_quarter']


In [81]:
select_columns = ['ID', 
                  'Year',
                  'Month',
                  'Temp', 
                  'Tmax', 
                  'Tmin',
                  'Bio01',
                  'Bio02',
                  'Bio03',
                  'Bio04',
                  'Bio05',
                  'Bio06',
                  'Bio07',
                  'Bio08',
                  'Bio09',
                  'Bio10',
                  'Bio11',
                  'Bio12',
                  'Bio13',
                  'Bio14',
                  'Bio15',
                  'Bio16',
                  'Bio17',
                  'Bio18',
                  'Bio19']

monthly_data[select_columns]

,ID,Year,Month,Temp,Tmax,Tmin,Bio01,Bio02,Bio03,Bio04,...,Bio10,Bio11,Bio12,Bio13,Bio14,Bio15,Bio16,Bio17,Bio18,Bio19
0,Abia,2016,1,301.822083,309.818440,292.485026,301.822083,17.333414,91.560896,1.820378,...,3018.220830,2414.785303,2.346305,0.373970,0.000051,124.993172,1.043558,0.249978,1.043558,0.249978
1,Abia,2016,2,303.404970,311.416049,293.600568,303.404970,17.815481,94.107335,1.820378,...,11816.964968,11210.368337,2.346305,0.373970,0.000051,124.993172,1.043558,0.249978,1.043558,0.249978
2,Abia,2016,3,300.428729,308.314380,296.658108,300.428729,11.656273,61.572335,1.820378,...,21130.255560,20529.004652,2.346305,0.373970,0.000051,124.993172,1.043558,0.249978,1.043558,0.249978
3,Abia,2016,4,300.422143,306.183258,297.173205,300.422143,9.010053,47.594118,1.820378,...,30142.919844,29540.985783,2.346305,0.373970,0.000051,124.993172,1.043558,0.249978,1.043558,0.249978
4,Abia,2016,5,299.306844,305.852426,295.684647,299.306844,10.167779,53.709615,1.820378,...,39421.432005,38823.974541,2.346305,0.373970,0.000051,124.993172,1.043558,0.249978,1.043558,0.249978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2237,Zamfara,2020,7,299.114053,307.019243,294.717056,299.114053,12.302186,40.142973,3.301192,...,496771.783479,496174.330117,0.810643,0.388939,0.000022,269.318887,0.688765,0.003023,0.688765,0.003023
2238,Zamfara,2020,8,297.520809,304.232498,293.674831,297.520809,10.557667,34.450474,3.301192,...,505994.928571,505401.352941,0.810643,0.388939,0.000022,269.318887,0.688765,0.003023,0.688765,0.003023
2239,Zamfara,2020,9,298.503096,305.857032,293.832603,298.503096,12.024429,39.236629,3.301192,...,514950.021440,514352.199511,0.810643,0.388939,0.000022,269.318887,0.688765,0.003023,0.688765,0.003023
2240,Zamfara,2020,10,299.503749,307.746636,291.504255,299.503749,16.242381,53.000127,3.301192,...,524234.637655,523635.529060,0.810643,0.388939,0.000022,269.318887,0.688765,0.003023,0.688765,0.003023


In [89]:
monthly_data[select_columns].to_csv('Bio_variable.csv', header=True, index=False)